In [1]:
pip install pandas scikit-learn numpy pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 51.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 54.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 48.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 34.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 23.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, LSTM, Dense, Dropout, BatchNormalization, Reshape
from keras.applications import DenseNet121
from keras.regularizers import l2
from keras_tuner import BayesianOptimization, HyperParameters
from keras.callbacks import EarlyStopping
from keras_tuner.engine.hypermodel import HyperModel

# Define the root directory where your image folders are located
root_directory = "PartB_DFU_dataset - Copy"

# Initialize lists to store image paths and corresponding class labels for both datasets
image_paths_ischaemia = []
categories_ischaemia = []
image_paths_infection = []
categories_infection = []

# Iterate over each class and its subdirectories
for class_name in ["Infection", "Ischaemia"]:
    for augmentation_type in ["Aug-Negative", "Aug-Positive"]:
        folder_path = os.path.join(root_directory, class_name, augmentation_type)
        category = f"{class_name.lower()}{'pov' if 'Positive' in augmentation_type else 'neg'}"
        
        # Iterate over image files in the current directory
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".jpg"):  # Assuming images are jpg format
                image_path = os.path.join(folder_path, file_name)
                if class_name == "Ischaemia":
                    image_paths_ischaemia.append(image_path)
                    categories_ischaemia.append("ischemia" if "Positive" in augmentation_type else "non-ischemia")
                elif class_name == "Infection":
                    image_paths_infection.append(image_path)
                    categories_infection.append("infection" if "Positive" in augmentation_type else "non-infection")

# Create DataFrames for each dataset
df_ischaemia = pd.DataFrame({"category": categories_ischaemia, "image_path": image_paths_ischaemia})
df_infection = pd.DataFrame({"category": categories_infection, "image_path": image_paths_infection})

# Label encoding for Ischaemia dataset
label_encoder_ischaemia = LabelEncoder()
df_ischaemia['Class_Label'] = label_encoder_ischaemia.fit_transform(df_ischaemia['category'])
print("Ischaemia Class Mapping:")
for class_label, numerical_label in zip(df_ischaemia['category'].unique(), df_ischaemia['Class_Label'].unique()):
    print(f"{class_label}: {numerical_label}")

# Label encoding for Infection dataset
label_encoder_infection = LabelEncoder()
df_infection['Class_Label'] = label_encoder_infection.fit_transform(df_infection['category'])
print("Infection Class Mapping:")
for class_label, numerical_label in zip(df_infection['category'].unique(), df_infection['Class_Label'].unique()):
    print(f"{class_label}: {numerical_label}")

# Shuffle both DataFrames
df_ischaemia = df_ischaemia.sample(frac=1).reset_index(drop=True)
df_infection = df_infection.sample(frac=1).reset_index(drop=True)

# Helper function to load and process images
def load_images(df):
    images = []
    target_labels = []   
    for index, row in df.iterrows():
        image = Image.open(row['image_path']).convert('RGB')
        image_array = np.array(image.resize((256, 256)))  # Resize image to fit DenseNet input size
        images.append(image_array)
        target_labels.append(row['Class_Label'])
    return np.array(images), np.array(target_labels)

# Load images for both datasets
images_ischaemia, target_labels_ischaemia = load_images(df_ischaemia)
images_infection, target_labels_infection = load_images(df_infection)

print("Shape of Ischaemia images array:", images_ischaemia.shape)
print("Shape of Ischaemia target labels array:", target_labels_ischaemia.shape)
print("Shape of Infection images array:", images_infection.shape)
print("Shape of Infection target labels array:", target_labels_infection.shape)

# Split the Ischaemia dataset
X_train_ischaemia, X_test_ischaemia, y_train_ischaemia, y_test_ischaemia = train_test_split(
    images_ischaemia, target_labels_ischaemia, test_size=0.3, random_state=42)
X_val_ischaemia, X_test_ischaemia, y_val_ischaemia, y_test_ischaemia = train_test_split(
    X_test_ischaemia, y_test_ischaemia, test_size=0.25, random_state=42)  # 0.25 * 0.3 = 0.075

# Split the Infection dataset
X_train_infection, X_test_infection, y_train_infection, y_test_infection = train_test_split(
    images_infection, target_labels_infection, test_size=0.3, random_state=42)
X_val_infection, X_test_infection, y_val_infection, y_test_infection = train_test_split(
    X_test_infection, y_test_infection, test_size=0.25, random_state=42)  # 0.25 * 0.3 = 0.075

print("Ischaemia Training set shape:", X_train_ischaemia.shape, y_train_ischaemia.shape)
print("Ischaemia Validation set shape:", X_val_ischaemia.shape, y_val_ischaemia.shape)
print("Ischaemia Test set shape:", X_test_ischaemia.shape, y_test_ischaemia.shape)
print("Infection Training set shape:", X_train_infection.shape, y_train_infection.shape)
print("Infection Validation set shape:", X_val_infection.shape, y_val_infection.shape)
print("Infection Test set shape:", X_test_infection.shape, y_test_infection.shape)


Ischaemia Class Mapping:
non-ischemia: 1
ischemia: 0
Infection Class Mapping:
non-infection: 1
infection: 0
Shape of Ischaemia images array: (9870, 256, 256, 3)
Shape of Ischaemia target labels array: (9870,)
Shape of Infection images array: (5890, 256, 256, 3)
Shape of Infection target labels array: (5890,)
Ischaemia Training set shape: (6909, 256, 256, 3) (6909,)
Ischaemia Validation set shape: (2220, 256, 256, 3) (2220,)
Ischaemia Test set shape: (741, 256, 256, 3) (741,)
Infection Training set shape: (4123, 256, 256, 3) (4123,)
Infection Validation set shape: (1325, 256, 256, 3) (1325,)
Infection Test set shape: (442, 256, 256, 3) (442,)


In [4]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
class MyHyperModel(HyperModel):

    def __init__(self):
        self.base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
        for layer in self.base_model.layers:
            layer.trainable = False

    def build(self, hp):
        model = Sequential([
            self.base_model,
            GlobalAveragePooling2D(),
            Reshape((8, 128)),
            LSTM(
                units=hp.Int('units_lstm1', min_value=16, max_value=32, step=8),
                return_sequences=True,
                kernel_regularizer=l2(hp.Float('l2_lstm1', min_value=0.001, max_value=0.01, step=0.001))
            ),
            Dropout(hp.Float('dropout_lstm1', min_value=0.4, max_value=0.5, step=0.1)),
            LSTM(
                units=hp.Int('units_lstm2', min_value=16, max_value=32, step=8),
                return_sequences=True,
                kernel_regularizer=l2(hp.Float('l2_lstm2', min_value=0.001, max_value=0.01, step=0.001))
            ),
            Dropout(hp.Float('dropout_lstm2', min_value=0.4, max_value=0.5, step=0.1)),
            LSTM(
                units=hp.Int('units_lstm3', min_value=16, max_value=32, step=8),
                kernel_regularizer=l2(hp.Float('l2_lstm3', min_value=0.001, max_value=0.01, step=0.001))
            ),
            BatchNormalization(),
            Dense(
                units=hp.Int('units_dense1', min_value=32, max_value=64, step=16),
                activation='relu',
                kernel_regularizer=l2(hp.Float('l2_dense1', min_value=0.001, max_value=0.01, step=0.001))
            ),
            Dropout(hp.Float('dropout_dense1', min_value=0.4, max_value=0.5, step=0.1)),
            BatchNormalization(),
            Dense(
                units=hp.Int('units_dense2', min_value=32, max_value=64, step=16),
                activation='relu',
                kernel_regularizer=l2(hp.Float('l2_dense2', min_value=0.001, max_value=0.01, step=0.001))
            ),
            Dropout(hp.Float('dropout_dense2', min_value=0.4, max_value=0.5, step=0.1)),
            BatchNormalization(),
            Dense(3, activation='softmax')
        ])

        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        return model

# Instantiate the HyperModel
hypermodel = MyHyperModel()


In [19]:
tuner = BayesianOptimization(
    hypermodel,
    objective='val_accuracy',
    max_trials=5,
    directory='infection_dir',
    project_name='infection_model'
)

In [20]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
tuner.search(X_train_infection, y_train_infection, epochs=50, validation_data=(X_val_infection, y_val_infection), callbacks=[reduce_lr, early_stopping])

Trial 5 Complete [00h 07m 22s]
val_accuracy: 0.7524528503417969

Best val_accuracy So Far: 0.7683019042015076
Total elapsed time: 00h 34m 45s


In [21]:
best_model_infection = tuner.get_best_models(num_models=1)[0]
test_loss_infection, test_accuracy_infection = best_model_infection.evaluate(X_test_infection, y_test_infection)
print(f"Infection Test Accuracy: {test_accuracy_infection}")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 44 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - accuracy: 0.7528 - loss: 0.5183
Infection Test Accuracy: 0.7556561231613159


W0000 00:00:1722269333.349117    1190 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722269333.349777    1190 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722269333.350424    1190 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722269333.351124    1190 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722269333.351810    1190 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722269333.352744    1190 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722269333.353562    1190 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722269333.354713    1190 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722269333.355923    1190 gp